In [1]:
from ilqr import iLQR
import gym
import numpy as np
import pandas as pd
import time

from aprl.agents import MujocoFiniteDiffDynamicsPerformance
from aprl.envs.mujoco_costs import get_cost
from experiments.common import set_seeds, make_env, fit_ilqr, \
                               on_iteration, receding, multi_evaluate

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Logging to /tmp/openai-2019-02-02-13-05-32-235612
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']


In [3]:
seed = 42
set_seeds(seed)

short_horizon = 50

env_name = 'InvertedPendulum-v2'
env, us_init = make_env(env_name, seed=seed, horizon=short_horizon)
dynamics = {'performance': MujocoFiniteDiffDynamicsPerformance(env)}
x0s = {k: dyn.get_state() for k, dyn in dynamics.items()}

costs = {k: get_cost(env_name) for k in dynamics.keys()}
ilqrs = {k: iLQR(dyn, costs[k], len(us_init))  
         for k, dyn in dynamics.items()}

# Open Loop

In [4]:
xs, us = fit_ilqr(ilqrs, x0s, us_init)

dict_keys(['performance'])
*** Fitting performance ***
iteration -1 converged 98.62615632028051 [-0.99884302  1.59168136 -2.18710654 -0.3509489 ] [-1.8908733]
iteration 0 accepted 55.08667724518653 [-0.33604464  1.63644166 -0.89895013 -0.73787982] [-0.7717686]
iteration 1 accepted 35.76847230409324 [-0.13107992  1.42375244 -0.2625448   5.35706429] [-0.18044272]
iteration 2 accepted 0.8206971341260431 [-0.01531407 -0.21129975 -0.23706047 -0.49392843] [-0.2426436]
iteration 3 accepted 0.001890966644626577 [0.01087307 0.00653197 0.02728976 0.01338608] [0.01333365]
iteration 4 accepted 0.0007434571828886829 [ 0.00736889 -0.0003039   0.00882465  0.00482244] [0.00054285]
iteration 5 accepted 0.0007434233101104847 [ 0.00736403 -0.00027667  0.00880865  0.00494782] [0.00054287]
iteration 6 accepted 0.0007434171863289613 [ 0.00735939 -0.00025028  0.00879359  0.00506883] [0.000543]
iteration 7 failed 0.0007434171863289614 [ 0.00735939 -0.00025028  0.00879359  0.00506883] [0.000543]
iteration 8 fa

# Closed Loop

In [7]:
for k, ilqr in ilqrs.items():
    underlying = 'receding_' + k
    dynamics[underlying] = dynamics[k]
    x0s[underlying] = x0s[k]
    xs[underlying], us[underlying] = receding(ilqr, x0s[k], us_init, seed=seed,
                                              horizon=5000, step_size=10)

iteration 0 x = [[-2.51713244e-03 -3.13228967e-04  8.41541642e-03 -3.61090414e-04]
 [-2.32565312e-03 -3.68820468e-04  1.07288464e-02 -5.16754091e-03]
 [-2.09773844e-03 -4.96166785e-04  1.20617967e-02 -7.56206461e-03]
 [-1.84926242e-03 -6.56889860e-04  1.27865811e-02 -8.51945364e-03]
 [-1.59002365e-03 -8.28407090e-04  1.31389109e-02 -8.64882617e-03]
 [-1.32602972e-03 -9.97855039e-04  1.32624879e-02 -8.31580526e-03]
 [-1.06098408e-03 -1.15817013e-03  1.32442101e-02 -7.73647094e-03]
 [-7.97193649e-04 -1.30570811e-03  1.31369475e-02 -7.03769756e-03]
 [-5.36111488e-04 -1.43882747e-03  1.29732805e-02 -6.29348464e-03]
 [-2.78661014e-04 -1.55704939e-03  1.27736339e-02 -5.54645954e-03]
 [-2.54307978e-05 -1.66055897e-03  1.25510901e-02 -4.82059295e-03]], u = [[ 0.01313067]
 [ 0.00789967]
 [ 0.00461073]
 [ 0.00255733]
 [ 0.00126136]
 [ 0.00042706]
 [-0.000123  ]
 [-0.00049485]
 [-0.00075221]
 [-0.00093381]]
iteration 10 x = [[-2.54307978e-05 -1.66055897e-03  1.25510901e-02 -4.82059295e-03]
 [ 2.2

iteration 160 x = [[-0.02019403 -0.0023054   0.03482677 -0.09538011]
 [-0.01967397 -0.00377123  0.01723194 -0.05168222]
 [-0.0194475  -0.0045204   0.0054476  -0.02352293]
 [-0.01942521 -0.00479247 -0.0031988  -0.0038605 ]
 [-0.0195588  -0.00471857 -0.01014755  0.01114527]
 [-0.01982213 -0.0043704  -0.01617788  0.02361424]
 [-0.02020112 -0.00378654 -0.02171725  0.03474774]
 [-0.02068837 -0.0029865  -0.02700745  0.04525976]
 [-0.02128034 -0.00197813 -0.03219116  0.05560267]
 [-0.02197577 -0.00076177 -0.03735462  0.06607997]
 [-0.02277473  0.00066737 -0.04254753  0.07690087]], u = [[-0.09363259]
 [-0.06358602]
 [-0.04745956]
 [-0.03880156]
 [-0.03415327]
 [-0.03167638]
 [-0.03040279]
 [-0.02981693]
 [-0.02962492]
 [-0.02963539]]
iteration 170 x = [[-0.02277473  0.00066737 -0.04254753  0.07690087]
 [-0.02346757  0.00182025 -0.02678138  0.03879226]
 [-0.02388932  0.00233192 -0.01542204  0.01262349]
 [-0.02410767  0.00238638 -0.0064307  -0.00702336]
 [-0.02415899  0.00208412  0.00128753 -0.0

iteration 320 x = [[-0.00622429 -0.01003011  0.12876614 -0.1369353 ]
 [-0.00388478 -0.0122106   0.10525574 -0.08175368]
 [-0.00194866 -0.0134715   0.08840238 -0.04474579]
 [-0.00031343 -0.01409432  0.07515071 -0.01780365]
 [ 0.00107648 -0.0142356   0.06386118  0.00349882]
 [ 0.0022515  -0.01398344  0.05365487  0.02160536]
 [ 0.00322856 -0.0133878   0.04405938  0.03789603]
 [ 0.00401729 -0.0124768   0.0348182   0.05318236]
 [ 0.00462336 -0.01126539  0.02578967  0.06797057]
 [ 0.00505024 -0.00976017  0.01689618  0.08259543]
 [ 0.00530026 -0.00796209  0.00809976  0.09728633]], u = [[-0.1266465 ]
 [-0.09252918]
 [-0.07428286]
 [-0.06449763]
 [-0.05919576]
 [-0.05625428]
 [-0.05454871]
 [-0.05348291]
 [-0.05272518]
 [-0.0520699 ]]
iteration 330 x = [[ 0.00530026 -0.00796209  0.00809976  0.09728633]
 [ 0.00558699 -0.00636859  0.02052784  0.06248843]
 [ 0.00609143 -0.00538061  0.0298835   0.03660449]
 [ 0.00676711 -0.0048607   0.03766093  0.01559696]
 [ 0.0075904  -0.00473367  0.04465075 -0.0

iteration 480 x = [[ 0.00791764 -0.00511059  0.0699558  -0.10710995]
 [ 0.00912532 -0.00678529  0.05087047 -0.0608802 ]
 [ 0.010006   -0.00768586  0.03723432 -0.02950501]
 [ 0.01064326 -0.00804146  0.02651595 -0.00626683]
 [ 0.0110818  -0.00797767  0.0173541   0.01251022]
 [ 0.01134539 -0.0075631   0.00901545  0.0288665 ]
 [ 0.01144651 -0.00683439  0.00110238  0.04396533]
 [ 0.01139149 -0.00580972 -0.00660242  0.05849716]
 [ 0.0111833  -0.00449609 -0.0142176   0.07289318]
 [ 0.01082312 -0.00289337 -0.02180423  0.08743404]
 [ 0.01031123 -0.00099667 -0.02939213  0.10231924]], u = [[-0.10210161]
 [-0.07415678]
 [-0.05936203]
 [-0.05159247]
 [-0.04756327]
 [-0.04552055]
 [-0.04452356]
 [-0.04405816]
 [-0.04382337]
 [-0.04365552]]
iteration 490 x = [[ 1.03112271e-02 -9.96673294e-04 -2.93921273e-02  1.02319237e-01]
 [ 9.89717219e-03  6.12433148e-04 -1.20676761e-02  5.90816423e-02]
 [ 9.77610492e-03  1.50323591e-03 -7.34311969e-05  3.03066066e-02]
 [ 9.86639824e-03  1.89843355e-03  9.08033342

iteration 640 x = [[ 0.15293998 -0.0180878   0.02053497 -0.07151231]
 [ 0.15302855 -0.01881234 -0.01160889 -0.0015437 ]
 [ 0.15259762 -0.01845268 -0.03145374  0.03725289]
 [ 0.15183731 -0.01748624 -0.04456657  0.0593228 ]
 [ 0.15085196 -0.01616825 -0.0539699   0.07251152]
 [ 0.14969921 -0.01463303 -0.0613129   0.08110248]
 [ 0.14841116 -0.01294867 -0.06750381  0.08745751]
 [ 0.14700575 -0.01114653 -0.07305     0.09289976]
 [ 0.14549302 -0.00923726 -0.07823819  0.09818279]
 [ 0.1438785  -0.0072197  -0.083229    0.10373811]
 [ 0.14216529 -0.00508594 -0.08810886  0.10981144]], u = [[-0.17873677]
 [-0.11382157]
 [-0.07806459]
 [-0.05813683]
 [-0.04684387]
 [-0.04030398]
 [-0.0364166 ]
 [-0.03402383]
 [-0.03246168]
 [-0.03133638]]
iteration 650 x = [[ 0.14216529 -0.00508594 -0.08810886  0.10981144]
 [ 0.1405431  -0.00326826 -0.07416113  0.07242337]
 [ 0.1391726  -0.00211461 -0.06292672  0.04327919]
 [ 0.13801265 -0.00149662 -0.05309616  0.01876714]
 [ 0.1370421  -0.00134186 -0.04397912 -0.0

...

iteration 4260 x = [[ 0.09880171 -0.0020859   0.03071931 -0.09982927]
 [ 0.09924987 -0.00366264  0.01414974 -0.05831952]
 [ 0.09941434 -0.00454128  0.00233137 -0.02984833]
 [ 0.09936782 -0.00492278 -0.00696149 -0.00850194]
 [ 0.09914882 -0.00491696 -0.0149232   0.00895316]
 [ 0.09877754 -0.00458352 -0.02219481  0.02430991]
 [ 0.0982643  -0.00395408 -0.02912251  0.03859117]
 [ 0.09761409 -0.00304397 -0.03589621  0.05240962]
 [ 0.0968289  -0.00185848 -0.04262266  0.06615847]
 [ 0.09590908 -0.00039626 -0.04936257  0.08010953]
 [ 0.094854    0.00134879 -0.05615134  0.09446811]], u = [[-0.08793433]
 [-0.06363161]
 [-0.05088611]
 [-0.04429869]
 [-0.0409744 ]
 [-0.03937268]
 [-0.03867831]
 [-0.03845855]
 [-0.03847394]
 [-0.03859232]]
iteration 4270 x = [[ 0.094854    0.00134879 -0.05615134  0.09446811]
 [ 0.09390489  0.00281373 -0.03881153  0.05249192]
 [ 0.09325669  0.00356656 -0.02604187  0.02308968]
 [ 0.0928394   0.00380125 -0.01570976  0.00057281]
 [ 0.09261577  0.0036239  -0.00666825 -0

iteration 4420 x = [[ 0.09165776 -0.01288502  0.04204461 -0.12241743]
 [ 0.0921901  -0.01462075  0.0112687  -0.05186607]
 [ 0.09222169 -0.01524521 -0.00806673 -0.01095277]
 [ 0.09192896 -0.01521359 -0.02118395  0.01392876]
 [ 0.09140791 -0.01477182 -0.03091047  0.03016986]
 [ 0.09071095 -0.01405189 -0.03878244  0.04180886]
 [ 0.08986673 -0.01312327 -0.04564055  0.05107721]
 [ 0.08889086 -0.01202069 -0.05195073  0.05923183]
 [ 0.08779165 -0.01075907 -0.05797529  0.06700033]
 [ 0.08657334 -0.00934174 -0.06386395  0.07481878]
 [ 0.08523771 -0.00776484 -0.06970746  0.0829711 ]], u = [[-0.16828786]
 [-0.10838692]
 [-0.07579767]
 [-0.05797826]
 [-0.04816622]
 [-0.04272102]
 [-0.03968126]
 [-0.03798117]
 [-0.03703085]
 [-0.03652105]]
iteration 4430 x = [[ 0.08523771 -0.00776484 -0.06970746  0.0829711 ]
 [ 0.08391336 -0.00632273 -0.06276125  0.06154866]
 [ 0.08272975 -0.00530045 -0.0556281   0.04093858]
 [ 0.08169057 -0.00468382 -0.04831319  0.02093714]
 [ 0.08079928 -0.0044623  -0.04083497  0

iteration 4580 x = [[ 0.00256163 -0.00460607  0.01499733 -0.08015094]
 [ 0.00266164 -0.00572699 -0.00494315 -0.03241532]
 [ 0.0024298  -0.0060705  -0.01821072 -0.00219797]
 [ 0.00196868 -0.00590527 -0.02788441  0.01858111]
 [ 0.00133343 -0.00537542 -0.03563281  0.03433944]
 [ 0.0005534  -0.00455663 -0.04236652  0.04752433]
 [-0.00035615 -0.00348634 -0.04858947  0.0595248 ]
 [-0.00138787 -0.00218    -0.05458534  0.07115647]
 [-0.00253879 -0.00064002 -0.06051315  0.08291233]
 [-0.00380839  0.00113907 -0.06645436  0.09508834]
 [-0.00519728  0.00316751 -0.07244557  0.10786935]], u = [[-0.10767289]
 [-0.07292413]
 [-0.05423685]
 [-0.04422538]
 [-0.03892207]
 [-0.0361995 ]
 [-0.0349055 ]
 [-0.03438667]
 [-0.03423364]
 [-0.03420743]]
iteration 4590 x = [[-0.00519728  0.00316751 -0.07244557  0.10786935]
 [-0.00646403  0.00488552 -0.05428472  0.06443483]
 [-0.00742001  0.00587936 -0.04134922  0.03527187]
 [-0.00814545  0.00636961 -0.03121917  0.01396416]
 [-0.00868339  0.00647784 -0.022591   -0

iteration 4730 x = [[ 0.07382746 -0.00550355 -0.00510861  0.09590746]
 [ 0.07389871 -0.00404084  0.01217945  0.05085153]
 [ 0.07425958 -0.0033267   0.02387367  0.02086025]
 [ 0.07482433 -0.00312948  0.03258049 -0.00095423]
 [ 0.07554729 -0.00332367  0.039703   -0.01835424]
 [ 0.07640438 -0.00384202  0.04599847 -0.03342226]
 [ 0.0773832  -0.00464989  0.05188009 -0.04734348]
 [ 0.07847779 -0.00573148  0.05757885 -0.06082614]
 [ 0.07968583 -0.00708257  0.0632278  -0.07432143]
 [ 0.08100709 -0.00870646  0.06890297 -0.08813213]
 [ 0.08244248 -0.01061157  0.07464399 -0.10246864]], u = [[0.09073504]
 [0.06188923]
 [0.04660535]
 [0.03858226]
 [0.0344401 ]
 [0.03237992]
 [0.03145068]
 [0.03114168]
 [0.03115476]
 [0.0312979 ]]
iteration 4740 x = [[ 0.08244248 -0.01061157  0.07464399 -0.10246864]
 [ 0.08371936 -0.01216227  0.05310386 -0.05313292]
 [ 0.08463121 -0.0129032   0.03811728 -0.02127653]
 [ 0.0852789  -0.01310372  0.02667309  0.00103472]
 [ 0.08571694 -0.0129109   0.01714443  0.01813766]

iteration 4880 x = [[ 0.00532318 -0.00139514  0.01007262 -0.09458113]
 [ 0.00535273 -0.00285182 -0.00706548 -0.05155922]
 [ 0.00509582 -0.00360037 -0.01859263 -0.02358387]
 [ 0.00463847 -0.00387214 -0.02712242 -0.00376945]
 [ 0.00402646 -0.00379229 -0.03406619  0.01164685]
 [ 0.00328381 -0.00342769 -0.04019102  0.02475179]
 [ 0.00242265 -0.00281252 -0.04592125  0.03673805]
 [ 0.00144842 -0.00196192 -0.05149986  0.04832332]
 [ 0.00036274 -0.00087928 -0.0570701   0.05996416]
 [-0.00083506  0.00043947 -0.06271318  0.07195707]
 [-0.00214681  0.00200326 -0.06846655  0.0844894 ]], u = [[-0.09110406]
 [-0.06212125]
 [-0.04675277]
 [-0.03870368]
 [-0.03460257]
 [-0.03265788]
 [-0.03192319]
 [-0.03187829]
 [-0.03219741]
 [-0.03264817]]
iteration 4890 x = [[-0.00214681  0.00200326 -0.06846655  0.0844894 ]
 [-0.00334971  0.00329225 -0.05187182  0.0448409 ]
 [-0.00426398  0.003908   -0.0395861   0.01700719]
 [-0.00495571  0.00403279 -0.02960709 -0.00435509]
 [-0.0054602   0.00376628 -0.02085544 -0

# Rollouts

In [6]:
multi_evaluate(env, dynamics, x0s, us)

performance
receding_performance


,rewards,lengths
performance,50.0,50
receding_performance,5000.0,5000
